<a href="https://colab.research.google.com/github/dotsnangles/Transformers-Text-Classification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q folium
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub

     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 
     |████████████████████████████████| 346 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.3 MB/s 
     |████████████████████████████████| 212 kB 56.7 MB/s 
     |████████████████████████████████| 140 kB 75.1 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 127 kB 49.4 MB/s 
     |████████████████████████████████| 271 kB 57.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 92.0 MB/s 
     |████████████████████████████████| 112 kB 75.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 re

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import datasets

from huggingface_hub import notebook_login

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy as np

In [3]:
# notebook_login()

xlm-roberta-base

klue/roberta-base

bert-base-multilingual-uncased

모델 3개
모델 당 케이폴드 5

### Data Load & K-Fold Interator Instantiation

In [4]:
!gdown -q 1K0v7liLb4ls5NMZrvrsEFQCfGqGduBSN

In [5]:
# df = pd.read_csv('/content/augmented_dataset.csv', index_col=False)
# df = df.drop(['Unnamed: 0'], axis=1)
# df.reset_index(inplace=True)
# X, y = df['title'], df['topic_idx']
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
# splitgen = skf.split(X, y)

### Function Declaration

In [18]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [19]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

### Global Configration

In [6]:
num_labels = 7

In [7]:
models = ['xlm-roberta-base', 'klue/roberta-base', 'bert-base-multilingual-uncased']

In [13]:
data_path = '/content/train_data.csv'

In [17]:
batch_size = 64
num_train_epochs = 1

### Fine Tuning and Model Saving

In [ ]:
for model_n in models:
    # Load Model & Tokenizer
    model_ckpt = model_n

    # Load Data & Prepare K-Fold
    df = pd.read_csv(data_path, index_col=False)
    # df = df.drop(['Unnamed: 0'], axis=1)
    # df.reset_index(inplace=True)
    X, y = df['title'], df['topic_idx']
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
    splitgen = skf.split(X, y)

    print(model_ckpt)

    foldNum = 1
    for train_index, test_index in splitgen:

        print(f'{model_ckpt} {foldNum}-Fold')

        # Model instantiation
        model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels,)
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

        # Tokenization
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        train = {
            'title': X_train.to_list(),
            'topic_idx': y_train.to_list()
        }
        test =  {
            'title': X_test.to_list(),
            'topic_idx': y_test.to_list()
        }
        train = datasets.Dataset.from_dict(train)
        train_encoded = train.map(tokenize, batched=True, batch_size=None)
        train_encoded = train_encoded.rename_column('topic_idx', 'label')

        test = datasets.Dataset.from_dict(test)
        test_encoded = test.map(tokenize, batched=True, batch_size=None)
        test_encoded = test_encoded.rename_column('topic_idx', 'label')

        # Training 
        logging_steps = len(train_encoded) // batch_size
        model_ckpt = model_ckpt.replace('/', '-')
        model_name = f"{model_ckpt}-tc-{foldNum}"
        training_args = TrainingArguments(output_dir=model_name,
                                            num_train_epochs=num_train_epochs,
                                            learning_rate=2e-5,
                                            per_device_train_batch_size=batch_size,
                                            per_device_eval_batch_size=batch_size,
                                            weight_decay=0.01,
                                            evaluation_strategy="epoch",
                                            disable_tqdm=False,
                                            logging_steps=logging_steps,
                                            push_to_hub=False,
                                            log_level="error")

        trainer = Trainer(model=model, args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=train_encoded,
                    eval_dataset=test_encoded,
                    tokenizer=tokenizer,
                    data_collator=data_collator)
        trainer.train()

        tokenizer.save_pretrained(f'saved/{model_name}')
        model.save_pretrained(f'saved/{model_name}')

        foldNum += 1

xlm-roberta-base
xlm-roberta-base 1-Fold


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.643800,0.397129,0.869127,0.867626


...

xlm-roberta-base 4-Fold


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss


In [16]:
import gc
gc.collect()

1221

In [ ]:
data_test = datasets.load_dataset('csv', data_files='/content/test_data.csv')['train']

Using custom data configuration default-4520c1e6af454092


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4520c1e6af454092/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data_test = data_test.remove_columns('index')

In [ ]:
encoded_test = data_test.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
encoded_test

Dataset({
    features: ['title', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9131
})

In [ ]:
preds_output = trainer.predict(encoded_data['test'])

In [ ]:
preds_output.metrics

{'test_accuracy': 0.8961778556565546,
 'test_f1': 0.8955660509094702,
 'test_loss': 0.31759849190711975,
 'test_runtime': 5.4783,
 'test_samples_per_second': 1666.753,
 'test_steps_per_second': 26.103}